In [17]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=tlt_encode
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/detectnet_v2
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tlt-samples/detectnet_v2

# Please define this local project directory that needs to be mapped to the TLT docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/detectnet_v2
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.

os.environ["LOCAL_PROJECT_DIR"] = "/home/ucit/UcitTrainer"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "detectnet_v2"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tlt-experiments/detectnet_v2/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

env: KEY=tlt_encode
env: NUM_GPUS=1
env: USER_EXPERIMENT_DIR=/workspace/tlt-experiments/detectnet_v2
env: DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data
env: SPECS_DIR=/workspace/tlt-experiments/detectnet_v2/specs
total 128
-rw-rw-r-- 1 ucit ucit  310 2月  25 21:15 detectnet_v2_tfrecords_kitti_trainval.txt
-rw-rw-r-- 1 ucit ucit 5675 2月  25 21:15 detectnet_v2_retrain_resnet18_kitti_qat.txt
-rw-rw-r-- 1 ucit ucit 2406 2月  25 21:15 detectnet_v2_inference_kitti_tlt.txt
-rw-rw-r-- 1 ucit ucit 2436 2月  25 21:15 detectnet_v2_inference_kitti_etlt.txt
-rw-rw-r-- 1 ucit ucit 2445 2月  25 21:15 detectnet_v2_inference_kitti_etlt_qat.txt
drwxr-xr-x 3 root root 4096 4月  14 13:23 tfrecords
-rw-rw-r-- 1 ucit ucit  310 4月  19 18:12 detectnet_v2_tfrecords_kitti_trainval_xuekitti.txt
-rw-rw-r-- 1 ucit ucit 5642 4月  20 10:41 detectnet_v2_train_resnet18_xuekitti.txt
-rw-rw-r-- 1 ucit ucit 5635 4月  20 11:18 detectnet_v2_retrain_resnet18_xuekitti.txt
-rw-rw-r-- 1 ucit ucit  310 4月  20 18:26 detectnet_v2_tf

In [18]:
# Mapping up the local directories to the TLT docker.
import json
mounts_file = os.path.expanduser("~/.tlt_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tlt-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [19]:
!cat ~/.tlt_mounts.json

{
    "Mounts": [
        {
            "source": "/home/ucit/UcitTrainer",
            "destination": "/workspace/tlt-experiments"
        },
        {
            "source": "/home/ucit/tlt_cv_samples_v1.0.2/detectnet_v2/specs",
            "destination": "/workspace/tlt-experiments/detectnet_v2/specs"
        }
    ]
}

In [20]:
# View the versions of the TLT launcher
!tlt info

Configuration of the TLT Instance
dockers: ['nvcr.io/nvidia/tlt-streamanalytics', 'nvcr.io/nvidia/tlt-pytorch']
format_version: 1.0
tlt_version: 3.0
published_date: 02/02/2021


In [5]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

Number of images in the train/val set. 14591
Number of labels in the train/val set. 14591
Number of images in the test set. 34


In [6]:
# Sample kitti label.
!cat $LOCAL_DATA_DIR/training/label_2/00001.txt

In [7]:
print("TFrecords conversion spec file for kitti training")
!cat $LOCAL_SPECS_DIR/detectnet_v2_tfrecords_ucit_trainval_xue.txt

TFrecords conversion spec file for kitti training
kitti_config {
  root_directory_path: "/workspace/tlt-experiments/data/training"
  image_dir_name: "image_2"
  label_dir_name: "label_2"
  image_extension: ".jpg"
  partition_mode: "random"
  num_partitions: 2
  val_split: 14
  num_shards: 10
}
image_directory_path: "/workspace/tlt-experiments/data/training"


In [8]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for kitti trainval dataset")


!tlt detectnet_v2 dataset_convert \
                  -d $SPECS_DIR/detectnet_v2_tfrecords_ucit_trainval_xue.txt \
                  -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_trainval/kitti_trainval

Converting Tfrecords for kitti trainval dataset
2021-05-10 13:26:12,231 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the ~/.tlt_mounts.json file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2021-05-10 05:26:16,484 - iva.detectnet_v2.dataio.build_converter - INFO - Instantiating a kitti converter
2021-05-10 05:26:16,484 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Creating output directory /workspace/tlt-experiments/data/tfrecords/kitti_trainval
2021-05-10 05:26:16,508 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Num images in
Train: 12549	Val: 2042
2021-05-10 05:26:16,508 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Validation data in partition 0. Hence, while choosing the validation

In [9]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/kitti_trainval/

total 11252
-rw-r--r-- 1 root root  163164 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 root root  160622 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 root root  157790 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 root root  162039 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 root root  160206 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 root root  161636 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 root root  161679 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 root root  162152 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 root root  158190 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 root root  164697 5月  10 13:26 kitti_trainval-fold-000-of-002-shard-00009-of-00010
-rw-r--r-- 1 root root  9908

In [21]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_reg_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_reg_linux.zip
--2021-05-12 09:46:46--  https://ngc.nvidia.com/downloads/ngccli_reg_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.33.29.28, 13.33.29.107, 13.33.29.117, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.33.29.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25011036 (24M) [application/zip]
Saving to: ‘/home/ucit/UcitTrainer/ngccli/ngccli_reg_linux.zip’

ngccli_reg_linux.zi 100%[===================>]  23.85M  9.29MB/s    in 2.6s    

2021-05-12 09:46:49 (9.29 MB/s) - ‘/home/ucit/UcitTrainer/ngccli/ngccli_reg_linux.zip’ saved [25011036/25011036]

Archive:  /home/ucit/UcitTrainer/ngccli/ngccli_reg_linux.zip
  inflating: /home/ucit/UcitTrainer/ngccli/ngc  
 extracting: /home/ucit/UcitTrainer/ngccli/ngc.md5  


In [11]:
# Create the target destination to download the model.
#!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

# for trafficCamera folder
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_trafficcamnet_Ucit/

In [12]:
#!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrained_trafficcamnet/tlt_pretrained_detectnet_v2_vresnet18

!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrained_trafficcamnet_Ucit/tlt_trafficcamnet_unpruned_v1.0

total 45384
-rw-rw-r-- 1 ucit ucit 46473000 8月   3  2020 resnet18_trafficcamnet.tlt


In [13]:
!tlt detectnet_v2 train -e $SPECS_DIR/detectnet_v2_train_resnet18_xueucit.txt \
                        -r $USER_EXPERIMENT_DIR/traffic_experiment_dir_unpruned \
                        -k $KEY \
                        -n resnet18_traffic_detector \
                        --gpus $NUM_GPUS

2021-05-10 13:27:34,324 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the ~/.tlt_mounts.json file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.





2021-05-10 05:27:38,867 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:117: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.


2021-05-10 05:27:38,867 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/horovod/tensorflow/__init__.py:143: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2021-05-10 05:27:39,214 [INFO] __main__: Loading experiment spec at /workspace/tlt-experiments/detectnet_v2/specs/detectnet_v2_train

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 544, 960)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 272, 480) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 272, 480) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 272, 480) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
block_1a_c

2021-05-10 05:27:45,935 [WARNING] tensorflow: Entity <bound method DriveNetTFRecordsParser.__call__ of <iva.detectnet_v2.dataloader.drivenet_dataloader.DriveNetTFRecordsParser object at 0x7f4212ad59b0>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Unable to locate the source code of <bound method DriveNetTFRecordsParser.__call__ of <iva.detectnet_v2.dataloader.drivenet_dataloader.DriveNetTFRecordsParser object at 0x7f4212ad59b0>>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
2021-05-10 05:27:45,948 [INFO] iva.detectnet_v2.dataloader.de

2021-05-10 05:27:47,865 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Serial augmentation enabled = False
2021-05-10 05:27:47,865 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Pseudo sharding enabled = False
2021-05-10 05:27:47,865 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: Max Image Dimensions (all sources): (0, 0)
2021-05-10 05:27:47,865 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: number of cpus: 20, io threads: 40, compute threads: 20, buffered batches: 4
2021-05-10 05:27:47,866 [INFO] modulus.blocks.data_loaders.multi_source_loader.data_loader: total dataset size 2042, number of sources: 1, batch size per gpu: 32, steps: 64
2021-05-10 05:27:47,873 [WARNING] tensorflow: Entity <bound method DriveNetTFRecordsParser.__call__ of <iva.detectnet_v2.dataloader.drivenet_dataloader.DriveNetTFRecordsParser object at 0x7f4212ad5a58>> could not be transformed and will be executed as-is. Please report t

INFO:tensorflow:Graph was finalized.
2021-05-10 05:27:50,021 [INFO] tensorflow: Graph was finalized.
INFO:tensorflow:Running local_init_op.
2021-05-10 05:27:51,150 [INFO] tensorflow: Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2021-05-10 05:27:51,552 [INFO] tensorflow: Done running local_init_op.
INFO:tensorflow:Saving checkpoints for step-0.
2021-05-10 05:27:55,761 [INFO] tensorflow: Saving checkpoints for step-0.
INFO:tensorflow:epoch = 0.0, loss = 0.05520262, step = 0
2021-05-10 05:28:10,811 [INFO] tensorflow: epoch = 0.0, loss = 0.05520262, step = 0
2021-05-10 05:28:10,813 [INFO] /usr/local/lib/python3.6/dist-packages/modulus/hooks/task_progress_monitor_hook.pyc: Epoch 0/120: loss: 0.05520 Time taken: 0:00:00 ETA: 0:00:00
2021-05-10 05:28:10,813 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 5.382
INFO:tensorflow:epoch = 0.033078880407124686, loss = 0.051217746, step = 13 (6.827 sec)
2021-05-10 05:28:17,639 [INFO] tensorflow: epoch = 0.0330788

...

INFO:tensorflow:epoch = 115.00000000000001, loss = 1.8391142e-05, step = 45195 (5.650 sec)
2021-05-10 10:27:44,027 [INFO] tensorflow: epoch = 115.00000000000001, loss = 1.8391142e-05, step = 45195 (5.650 sec)
2021-05-10 10:27:44,027 [INFO] /usr/local/lib/python3.6/dist-packages/modulus/hooks/task_progress_monitor_hook.pyc: Epoch 115/120: loss: 0.00002 Time taken: 0:02:10.657017 ETA: 0:10:53.285083
2021-05-10 10:27:45,330 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 78.182
INFO:tensorflow:global_step/sec: 2.6303
2021-05-10 10:27:45,982 [INFO] tensorflow: global_step/sec: 2.6303
INFO:tensorflow:epoch = 115.04325699745547, loss = 2.1410833e-05, step = 45212 (5.543 sec)
2021-05-10 10:27:49,570 [INFO] tensorflow: epoch = 115.04325699745547, loss = 2.1410833e-05, step = 45212 (5.543 sec)
2021-05-10 10:27:53,499 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.932
INFO:tensorflow:epoch = 115.08651399491094, loss = 1.836018e-05, step = 45229 (5.558 sec)
2021-

INFO:tensorflow:epoch = 116.07888040712469, loss = 1.7233044e-05, step = 45619 (5.549 sec)
2021-05-10 10:30:02,633 [INFO] tensorflow: epoch = 116.07888040712469, loss = 1.7233044e-05, step = 45619 (5.549 sec)
2021-05-10 10:30:04,274 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 98.047
INFO:tensorflow:global_step/sec: 3.0593
2021-05-10 10:30:06,239 [INFO] tensorflow: global_step/sec: 3.0593
INFO:tensorflow:epoch = 116.12213740458016, loss = 1.5301339e-05, step = 45636 (5.571 sec)
2021-05-10 10:30:08,205 [INFO] tensorflow: epoch = 116.12213740458016, loss = 1.5301339e-05, step = 45636 (5.571 sec)
2021-05-10 10:30:12,483 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.452
INFO:tensorflow:epoch = 116.16539440203563, loss = 2.1462685e-05, step = 45653 (5.583 sec)
2021-05-10 10:30:13,787 [INFO] tensorflow: epoch = 116.16539440203563, loss = 2.1462685e-05, step = 45653 (5.583 sec)
INFO:tensorflow:global_step/sec: 3.05182
2021-05-10 10:30:19,019 [INFO] tensor

2021-05-10 10:32:23,410 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.766
INFO:tensorflow:global_step/sec: 3.05196
2021-05-10 10:32:26,682 [INFO] tensorflow: global_step/sec: 3.05196
INFO:tensorflow:epoch = 117.20356234096693, loss = 2.2496659e-05, step = 46061 (5.567 sec)
2021-05-10 10:32:27,337 [INFO] tensorflow: epoch = 117.20356234096693, loss = 2.2496659e-05, step = 46061 (5.567 sec)
2021-05-10 10:32:31,582 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.896
INFO:tensorflow:epoch = 117.2468193384224, loss = 1.5761894e-05, step = 46078 (5.570 sec)
2021-05-10 10:32:32,907 [INFO] tensorflow: epoch = 117.2468193384224, loss = 1.5761894e-05, step = 46078 (5.570 sec)
INFO:tensorflow:epoch = 117.29007633587787, loss = 1.7186085e-05, step = 46095 (5.544 sec)
2021-05-10 10:32:38,451 [INFO] tensorflow: epoch = 117.29007633587787, loss = 1.7186085e-05, step = 46095 (5.544 sec)
INFO:tensorflow:global_step/sec: 3.06048
2021-05-10 10:32:39,426 [INFO] tensor

INFO:tensorflow:epoch = 118.27989821882953, loss = 1.9703679e-05, step = 46484 (5.337 sec)
2021-05-10 10:34:45,702 [INFO] tensorflow: epoch = 118.27989821882953, loss = 1.9703679e-05, step = 46484 (5.337 sec)
INFO:tensorflow:global_step/sec: 3.04358
2021-05-10 10:34:47,012 [INFO] tensorflow: global_step/sec: 3.04358
2021-05-10 10:34:50,590 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 98.187
INFO:tensorflow:epoch = 118.323155216285, loss = 1.753842e-05, step = 46501 (5.551 sec)
2021-05-10 10:34:51,253 [INFO] tensorflow: epoch = 118.323155216285, loss = 1.753842e-05, step = 46501 (5.551 sec)
INFO:tensorflow:epoch = 118.36641221374046, loss = 2.0461246e-05, step = 46518 (5.534 sec)
2021-05-10 10:34:56,787 [INFO] tensorflow: epoch = 118.36641221374046, loss = 2.0461246e-05, step = 46518 (5.534 sec)
2021-05-10 10:34:58,731 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 98.279
INFO:tensorflow:global_step/sec: 3.06951
2021-05-10 10:34:59,718 [INFO] tensorflow

INFO:tensorflow:global_step/sec: 3.05821
2021-05-10 10:37:07,369 [INFO] tensorflow: global_step/sec: 3.05821
2021-05-10 10:37:09,646 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.903
INFO:tensorflow:epoch = 119.40203562340967, loss = 2.0607713e-05, step = 46925 (5.575 sec)
2021-05-10 10:37:09,976 [INFO] tensorflow: epoch = 119.40203562340967, loss = 2.0607713e-05, step = 46925 (5.575 sec)
INFO:tensorflow:epoch = 119.44529262086515, loss = 1.8124567e-05, step = 46942 (5.541 sec)
2021-05-10 10:37:15,518 [INFO] tensorflow: epoch = 119.44529262086515, loss = 1.8124567e-05, step = 46942 (5.541 sec)
2021-05-10 10:37:17,812 [INFO] modulus.hooks.sample_counter_hook: Train Samples / sec: 97.961
INFO:tensorflow:global_step/sec: 3.06128
2021-05-10 10:37:20,109 [INFO] tensorflow: global_step/sec: 3.06128
INFO:tensorflow:epoch = 119.48854961832062, loss = 1.910697e-05, step = 46959 (5.564 sec)
2021-05-10 10:37:21,082 [INFO] tensorflow: epoch = 119.48854961832062, loss = 1.910697

In [22]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/traffic_experiment_dir_unpruned/weights

Model for each epoch:
---------------------
total 43M
-rw-r--r-- 1 root root 43M 5月  10 18:38 resnet18_traffic_detector.tlt


## 5. Evaluate the trained model <a class="anchor" id="head-5"></a>

In [27]:
# Creating a new directory for the output tfrecords dump.
print("Begin to Visualization the testing image_2!!!!!")
!tlt detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_ucit_tlt.txt \
                            -o $USER_EXPERIMENT_DIR/Outresults \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

Begin to Visualization the testing image_2!!!!!
2021-05-12 10:32:29,194 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the ~/.tlt_mounts.json file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2021-05-12 02:32:33,630 [INFO] __main__: Overlain images will be saved in the output path.
2021-05-12 02:32:33,630 [INFO] iva.detectnet_v2.inferencer.build_inferencer: Constructing inferencer

2021-05-12 02:32:33,631 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/detectnet_v2/inferencer/tlt_inferencer.py:83: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.Confi

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']


def visualize_images(image_dir, num_cols=4, imgfrom=0,imgto=12):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(imgfrom) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[imgfrom:imgto]):
        print("idx:",idx)
        print("img_path:",img_path)
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        print("col_id:",col_id)
        print("row_id:",row_id)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
 #Visualizing the first 12 images.
OUTPUT_PATH = 'Outresults/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
frompic = 58 # number of images to visualize.
topic = 70
visualize_images(OUTPUT_PATH, num_cols=COLS, imgfrom=frompic, imgto=topic)

## 6. Prune the trained model <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Applicable for resnets and mobilenets`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

*For some internal studies, we have noticed that a pth value of 0.01 is a good starting point for detectnet_v2 models.*

In [ ]:
# Create an output directory if it doesn't exist.
#!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

#!mkdir -p $LOCAL_EXPERIMENT_DIR/traffic_dir_pruned
!mkdir -p $LOCAL_EXPERIMENT_DIR/traffic_dir_pruned_UA

In [ ]:
!tlt detectnet_v2 prune \
                  -m $USER_EXPERIMENT_DIR/traffic_experiment_dir_unpruned/weights/resnet18_traffic_detector.tlt \
                  -o $USER_EXPERIMENT_DIR/traffic_dir_pruned_UA/resnet18_traffic_detector_pruned.tlt \
                  -eq union \
                  -pth 0.0000052 \
                  -k $KEY

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/traffic_dir_pruned_UA/

## 7. Retrain the pruned model <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in mAP, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: DetectNet_v2 now supports Quantization Aware Training, to help with optmizing the model. By default, the training in the cell below doesn't run the model with QAT enabled. For information on training a model with QAT, please refer to the cells under [section 11](#head-11)*

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_xue.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tlt detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_xue.txt \
                        -r $USER_EXPERIMENT_DIR/traffic_dir_retrained_pruned_UA \
                        -k $KEY \
                        -n resnet18_traffic_detector_pruned \
                        --gpus $NUM_GPUS

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/traffic_dir_retrained_pruned_UA/weights

## 8. Evaluate the retrained model <a class="anchor" id="head-8"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tlt detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_xue.txt \
                           -m $USER_EXPERIMENT_DIR/traffic_dir_retrained_pruned_UA/weights/resnet18_traffic_detector_pruned.tlt \
                           -k $KEY

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `inference` tool to generate inferences on the trained models. To render bboxes from more classes, please edit the spec file `detectnet_v2_inference_kitti_tlt.txt` to include all the classes you would like to visualize and edit the rest of the file accordingly.

In [ ]:
# Running inference for detection on n images
!tlt detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_tlt_UA.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

The `inference` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/tlt_infer_testing/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/tlt_infer_testing/labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `inference` command with the path to the image.*

In [ ]:
# Simple grid visualizer
#!pip3 install matplotlib==3.3.3
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']


def visualize_images(image_dir, num_cols=4, imgfrom=0,imgto=12):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(imgfrom) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[imgfrom:imgto]):
        print("idx:",idx)
        print("img_path:",img_path)
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        print("col_id:",col_id)
        print("row_id:",row_id)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
frompic = 58 # number of images to visualize.
topic = 70
visualize_images(OUTPUT_PATH, num_cols=COLS, imgfrom=frompic, imgto=topic)

## 10. Model Export <a class="anchor" id="head-10"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/traffic_final_UA
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "traffic_final_UA/resnet18_traffic_detector.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tlt detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/traffic_dir_retrained_pruned_UA/weights/resnet18_traffic_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/traffic_final_UA/resnet18_trafficcamnet_UA_Xue.etlt \
                  -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/traffic_final_UA

### A. Int8 Optimization <a class="anchor" id="head-10-1"></a>
DetectNet_v2 model supports int8 inference mode in TensorRT. 
In order to use int8 mode, we must calibrate the model to run 8-bit inferences -

* Generate calibration tensorfile from the training data using detectnet_v2 calibration_tensorfile
* Use tlt-export to generate int8 calibration table.

*Note: For this example, we generate a calibration tensorfile containing 10 batches of training data.
Ideally, it is best to use atleast 10-20% of the training data to do so. The more data provided during calibration, the closer int8 inferences are to fp32 inferences.*

*Note: If the model was trained with QAT nodes available, please refrain from using the post training int8 optimization as mentioned below. Please export the model in int8 mode (using the arg `--data_type int8`) with just the path to the calibration cache file (using the argument `--cal_cache_file`)*

In [ ]:
!tlt detectnet_v2 calibration_tensorfile -e $SPECS_DIR/detectnet_v2_retrain_resnet18_xue.txt \
                                         -m 10 \
                                         -o $USER_EXPERIMENT_DIR/traffic_final_UA/calibration.tensor

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/traffic_final_UA/resnet18__traffic_detector.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/traffic_final_UA/calibration.bin
!tlt detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/traffic_dir_retrained_pruned_UA/weights/resnet18_traffic_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/traffic_final_UA/resnet18_trafficcamnet.etlt \
                  -k $KEY  \
                  --cal_data_file $USER_EXPERIMENT_DIR/traffic_final_UA/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 4 \
                  --max_batch_size 4\
                  --engine_file $USER_EXPERIMENT_DIR/traffic_final_UA/resnet18_trafficcamnet.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/traffic_final_UA/calibration.bin \
                  --verbose

### B. Generate TensorRT engine <a class="anchor" id="head-10-2"></a>
Verify engine generation using the `tlt-converter` utility included with the docker.

The `tlt-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tlt-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the converter for jetson from the dev zone link [here](https://developer.nvidia.com/tlt-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
!tlt tlt-converter $USER_EXPERIMENT_DIR/traffic_final/resnet18_traffic_detector.etlt \
                   -k $KEY \
                    -c $USER_EXPERIMENT_DIR/traffic_final/calibration.bin \
                   -o output_cov/Sigmoid,output_bbox/BiasAdd \
                   -d 3,384,1248 \
                   -i nchw \
                   -m 64 \
                   -t int8 \
                   -e $USER_EXPERIMENT_DIR/traffic_final/resnet18_traffic_detector.trt \
                   -b 4

## 11. Verify Deployed Model <a class="anchor" id="head-11"></a>
Verify the exported model by visualizing inferences on TensorRT.
In addition to running inference on a `.tlt` model in [step 9](#head-9), the `inference` tool is also capable of consuming the converted `TensorRT engine` from [step 10.B](#head-10-2).

*If after int-8 calibration the accuracy of the int-8 inferences seem to degrade, it could be because the there wasn't enough data in the calibration tensorfile used to calibrate thee model or, the training data is not entirely representative of your test images, and the calibration maybe incorrect. Therefore, you may either regenerate the calibration tensorfile with more batches of the training data and recalibrate the model, or calibrate the model on a few images from the test set. This may be done using `--cal_image_dir` flag in the `export` tool. For more information, please follow the instructions in the USER GUIDE.

### A. Inference using TensorRT engine <a class="anchor" id="head-11-1"></a>

In [ ]:
!tlt detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt.txt \
                            -o $USER_EXPERIMENT_DIR/etlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'etlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 11. QAT workflow <a class="anchor" id="head-12"></a>
This section delves into the newly enabled Quantization Aware Training feature with DetectNet_v2. The workflow defined below converts a pruned model from section [5](#head-5) to enable QAT and retrain this model to while accounting the noise introduced due to quantization in the forward pass. 

### A. Convert pruned model to QAT and retrain <a class="anchor" id="head-12-1"></a>
All detectnet models, unpruned and pruned models can be converted to QAT models by setting the `enable_qat` parameter in the `training_config` component of the spec file to `true`.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to convert the
# pretrained model to qat mode by setting the enable_qat
# parameter.
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt

In [ ]:
!tlt detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat \
                        -k $KEY \
                        -n resnet18_detector_pruned_qat \
                        --gpus $NUM_GPUS

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

### B. Evaluate QAT converted model <a class="anchor" id="head-12-2"></a>
This section evaluates a QAT enabled pruned retrained model. The mAP of this model should be comparable to that of the pruned retrained model without QAT. However, due to quantization, it is possible sometimes to see a drop in the mAP value for certain datasets.

In [ ]:
!tlt detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                           -k $KEY \
                           -f tlt

### C. Export QAT trained model to int8 <a class="anchor" id="head-12-3"></a>
Export a QAT trained model to TensorRT parsable model. This command generates an .etlt file from the trained model and the serializes corresponding int8 scales as a TRT readable calibration cache file.

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin
!tlt detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt \
                  -k $KEY  \
                  --data_type int8 \
                  --batch_size 64 \
                  --max_batch_size 64\
                  --engine_file $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin \
                  --verbose

### D. Evaluate a QAT trained model using the exported TensorRT engine <a class="anchor" id="head-12-4"></a>
This section evaluates a QAT enabled pruned retrained model using the TensorRT int8 engine that was exported in [Section C](#head-12-3). Please note that there maybe a slight difference (~0.1-0.5%) in the mAP from [Section B](#head-12-2), oweing to some differences in the implementation of quantization in TensorRT.

*Note: The TensorRT evaluator might be slightly slower than the TLT evaluator here, because the evaluation dataloader is pinned to the CPU to avoid any clashes between TensorRT and TLT instances in the GPU. Please note that this tool was not intended and has not been developed for profiling the model. It is just a means to qualitatively analyse the model.*

*Please use native TensorRT or DeepStream for the most optimized inferences.*

In [ ]:
!tlt detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                           -f tensorrt

### E. Inference using QAT engine <a class="anchor" id="head-12-5"></a>
Run inference and visualize detections on test images, using the exported TensorRT engine from [Section C](#head-12-3).

In [ ]:
!tlt detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt_qat.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing_qat \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'tlt_infer_testing_qat/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)